# DL Assignment 2

Ishaan Reddy
SCDS
210200019

In [1]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import keras
tf.random.set_seed(42)

import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt
%matplotlib inline

import glob
import PIL
from PIL import Image

2.10.1


In [2]:
imgFiles = glob.glob(r'C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy/*/*.jpg')


for item in imgFiles:
    print(item)

C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (1).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (10).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (11).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (12).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (13).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (14).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (15).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (16).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (17).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning

In [3]:
X = []
y = []

for fName in imgFiles:

  # Prepare the dataset and populate X and y
    X_i = Image.open(fName) 
    X_i = X_i.resize((299,299)) # To make them approriate to Xception model when using Transfer Learning
    X_i = np.array(X_i) / 255.0 # Normalize to range 0.0 to 1.0 (not stretching, only scaling)

    X.append(X_i)

    label = fName.split("\\")
    y_i = label[-2] 

    y.append(y_i)


In [4]:
from sklearn.preprocessing import LabelEncoder
lEncoder = LabelEncoder()
y = lEncoder.fit_transform(y)

print(set(y))
print(lEncoder.classes_)

X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

{0, 1, 2, 3, 4, 5, 6}
['african-wildcat' 'blackfoot-cat' 'chinese-mountain-cat' 'domestic-cat'
 'european-wildcat' 'jungle-cat' 'sand-cat']
(519, 299, 299, 3)
(519,)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                    stratify=y, random_state=42)

print("X_train_shape: {}".format(X_train.shape))
print("X_test_shape: {}".format(X_test.shape))

X_train_shape: (389, 299, 299, 3)
X_test_shape: (130, 299, 299, 3)


In [6]:
mu = X_train.mean()
std = X_train.std()

X_train_std = (X_train-mu)/std
X_test_std = (X_test-mu)/std

a. Total number of images in the dataset = 519

b. Number of images present per class
    Class: african-wildcat, Count: 91
    Class: blackfoot-cat, Count: 79
    Class: chinese-mountain-cat, Count: 42
    Class: domestic-cat, Count: 64
    Class: european-wildcat, Count: 85
    Class: jungle-cat, Count: 86
    Class: sand-cat, Count: 72 
    
c. Total number of images used for training and testing.
    Train = 389
    Test = 130

## Transfer Learning

### Model 3: EfficientNetV2B0

In [7]:
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D


base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(299, 299, 3))


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.15)(x)  
predictions = Dense(7, activation='softmax')(x) 

model_TL3 = Model(inputs=[base_model.input], outputs=[predictions])

In [8]:
model_TL3.compile(loss='sparse_categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

callbacks_TL3 = [
             keras.callbacks.ModelCheckpoint("bestTL3.h5",
                                             monitor='val_accuracy',
                                             save_weights_only=True,
                                             save_best_only=True)]

history_TL3 = model_TL3.fit(x = X_train_std, y = y_train, epochs=10,
                          validation_split=0.1, batch_size=16, callbacks=callbacks_TL3)

Epoch 1/10
22/22 [==============================] - 33s 987ms/step - loss: 1.9862 - accuracy: 0.1714 - val_loss: 1.9072 - val_accuracy: 0.1282
Epoch 2/10
22/22 [==============================] - 19s 884ms/step - loss: 1.9400 - accuracy: 0.1686 - val_loss: 1.9189 - val_accuracy: 0.1538
Epoch 3/10
22/22 [==============================] - 18s 811ms/step - loss: 1.9194 - accuracy: 0.1771 - val_loss: 1.8720 - val_accuracy: 0.1795
Epoch 4/10
22/22 [==============================] - 17s 778ms/step - loss: 1.9079 - accuracy: 0.1914 - val_loss: 1.8985 - val_accuracy: 0.1282
Epoch 5/10
22/22 [==============================] - 19s 851ms/step - loss: 1.9243 - accuracy: 0.1829 - val_loss: 1.8560 - val_accuracy: 0.2564
Epoch 6/10
22/22 [==============================] - 17s 795ms/step - loss: 1.9153 - accuracy: 0.1971 - val_loss: 1.8816 - val_accuracy: 0.1795
Epoch 7/10
22/22 [==============================] - 17s 788ms/step - loss: 1.8912 - accuracy: 0.1943 - val_loss: 1.8677 - val_accuracy: 0.1795

### Evaluate the Model 

In [9]:
testLoss_TL, testAccuracy_TL = model_TL3.evaluate(x = X_test_std, y = y_test)
print("Test-loss: %f, Test-accuracy: %f" % (testLoss_TL, testAccuracy_TL))

5/5 [==============================] - 6s 993ms/step - loss: 1.8713 - accuracy: 0.1923
Test-loss: 1.871283, Test-accuracy: 0.192308


### Update with best weights 

In [10]:
model_TL3.load_weights("bestTL3.h5")

testLoss_TL, testAccuracy_TL = model_TL3.evaluate(x = X_test_std, y = y_test)

print("Test-loss: %f, Test-accuracy: %f" % (testLoss_TL, testAccuracy_TL))

5/5 [==============================] - 6s 1s/step - loss: 1.8863 - accuracy: 0.1923
Test-loss: 1.886329, Test-accuracy: 0.192308


In [11]:
model_TL3.save('/content/drive/MyDrive/DL_Models/03_Xception_TransferLearning_Best_Model.h5')

In [12]:
from numpy import save

save('/content/drive/MyDrive/DL_Models/X_train_std.npy', X_train_std)
save('/content/drive/MyDrive/DL_Models/X_test_std.npy', X_test_std)

save('/content/drive/MyDrive/DL_Models/y_train.npy', y_train)
save('/content/drive/MyDrive/DL_Models/y_test.npy', y_test)